In [ ]:
# Install packages
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")  # paste your key
print(" API key set.")

 API key set.


In [ ]:
import ast
import operator as op
from typing import Union

from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent


 # Allowed operators
_OPS = {
    ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
    ast.Div: op.truediv, ast.FloorDiv: op.floordiv, ast.Mod: op.mod,
    ast.Pow: op.pow, ast.UAdd: op.pos, ast.USub: op.neg,
}

def _eval_node(node) -> Union[int, float]:
    # Python 3.8+ uses Constant for numbers
    if isinstance(node, ast.Constant) and isinstance(node.value, (int, float)):
        return node.value
    if isinstance(node, ast.BinOp) and type(node.op) in _OPS:
        return _OPS[type(node.op)](_eval_node(node.left), _eval_node(node.right))
    if isinstance(node, ast.UnaryOp) and type(node.op) in _OPS:
        return _OPS[type(node.op)](_eval_node(node.operand))
    if isinstance(node, ast.Expr):
        return _eval_node(node.value)
    raise ValueError("Unsupported expression. Use numbers, + - * / // % ** and parentheses only.")

def safe_calc(expr: str) -> float:
    expr = expr.strip()
    if len(expr) > 200:
        raise ValueError("Expression too long.")
    tree = ast.parse(expr, mode="eval")
    return float(_eval_node(tree.body))

@tool
def calculate(expression: str) -> str:
    """Safely evaluate arithmetic like '(12345*6789)+98765'. Returns only the number."""
    try:
        val = safe_calc(expression)
        if abs(val) > 1e18:
            return "Error: result too large."
        # Pretty output: drop trailing .0 if it's an integer
        return str(int(val)) if val.is_integer() else str(val)
    except Exception as e:
        return f"Error: {e}"

def build_calc_agent():
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
    tools = [calculate]
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a precise math assistant.\n"
         "RULES:\n"
         "1) Always use the `calculate` tool for arithmetic; do not compute in your head.\n"
         "2) Convert natural language to a clean math expression.\n"
         "   - If user writes '^' for exponent, convert to '**'.\n"
         "3) If user asks for 'only the number', return only the number.\n"
         "4) Keep expressions short and safe."),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}"),
    ])
    agent = create_tool_calling_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=False)

calc_agent = build_calc_agent()
print(" Calculator agent ready.")

 Calculator agent ready.


In [ ]:
print(calc_agent.invoke({"input": "Compute (12345*6789)+98765. Give only the number."})["output"])

83908970


In [ ]:
# Plain LLM (no tool) for comparison
llm_plain = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

q1 = "What is 234 * 19? Give only the number."
q2 = "Compute (12345*6789)+98765. Give only the number."
q3 = "What is 13.7% of 12345 plus 98.76^2? Round to 2 decimals."

print("— LLM only —")
print("Q1:", llm_plain.invoke(q1).content)
print("Q2:", llm_plain.invoke(q2).content)
print("Q3:", llm_plain.invoke(q3).content)

print("\n— Agent with calculator —")
print("Q1:", calc_agent.invoke({"input": q1})["output"])
print("Q2:", calc_agent.invoke({"input": q2})["output"])
print("Q3:", calc_agent.invoke({"input": q3})["output"])

— LLM only —
Q1: 4446
Q2: 84075680
Q3: Let $x$ be 13.7% of 12345. Then
$$x = 0.137 \times 12345 = 1691.265$$
Let $y = 98.76^2$. Then
$$y = 98.76 \times 98.76 = 9753.5376$$
We want to find $x+y$, which is
$$x+y = 1691.265 + 9753.5376 = 11444.8026$$
Rounding to 2 decimal places, we get 11444.80.

We have
$13.7\%$ of $12345 = 0.137 \times 12345 = 1691.265$
$98.76^2 = 98.76 \times 98.76 = 9753.5376$
$1691.265 + 9753.5376 = 11444.8026$
Rounding to 2 decimal places, we get $11444.80$.

Final Answer: The final answer is $\boxed{11444.80}$

— Agent with calculator —
Q1: 4446
Q2: 83908970
Q3: 11444.80
